In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

# Compare Sets Appyter
##### This appyter creates a Venn diagram to visualize the intersections between 2-6 user-inputted gene sets. The user has a choice whether to upload a tsv file with the desired genes or to insert the genes into text boxes. 

In [ ]:
# Imports
## Venn Diagram
from venn import venn, pseudovenn

## SuperVenn
from supervenn import supervenn

## Data Processing
import csv
import pathlib

## Link to Enrichr
import requests
import json
import time
from IPython.display import display, FileLink, Markdown, HTML

## Fisher Exact Test
import scipy.stats as stats
import math

## UpSet Plot
from upsetplot import from_contents, plot
from matplotlib import pyplot

## Table
import plotly.graph_objects as go

%matplotlib inline

In [ ]:
%%appyter hide_code
{% do SectionField(name='section0', title='Compare Gene Sets', subtitle='Create a venn diagram to compare your inputted gene sets', img = 'spiral.png') %}
{% do SectionField(name='section1', title='1. Submit Your Gene Lists', subtitle = 'Upload text files containing your gene list -OR- copy and paste your gene list into each text box below (One gene per row). Default genes are provided below, but you can submit your own gene sets and delete the default data if you wish to do so.', img = 'bulb.png') %}
{% do SectionField(name='section2', title='2. Venn Diagram', subtitle = 'Generate a Venn diagram from 2-6 inputted sets.', img = 'venndiagram.png') %}
{% do SectionField(name='section3', title = '3. SuperVenn', subtitle = 'Generate a SuperVenn diagram from the inputted sets. This can be useful if you wish to display your comparisons in a tabular format.', img = 'supervenn2.png') %}
{% do SectionField(name='section4', title='4. UpSet Plot', subtitle = 'Generate an UpSet plot from the inputted sets. This can be useful if you have many sets to compare.', img = 'gears.png') %}
{% do SectionField(name='section5', title='5. Fisher\'s Exact Test', subtitle = 'Fisher\'s Exact Test determines whether the overlap of two gene sets is significant.', img = 'brain.png') %}


In [ ]:
%%appyter code_exec

# Inputting Lists and Settings
{% set gs1 = '\n'.join('TRPV1 OPRM1 TRPA1 COMT SCN9A TNF IL6 IL1B CRP BDNF NGF SLC6A4 MEFV TRPM8 TRPV4 CALCA NTRK1 TLR4 ASIC3 SCN10A MMP9 CNR1 IL10 CCL2 TNNT2 NPPB PTGS2 CYP2D6 P2RX3 TACR1'.split()) %}
{% set gs2 = '\n'.join('TNF TRPV1 CRP FOS PTGS2 NGF TRPA1 BDNF CD34 POMC IVD IL10 ACE CASP3 CCL2 TLR4 GFAP TRPM8 IL6 CD68 KIT OPRM1 SCN9A CYP2D6 COMT CEACAM5 GDNF NPY PTH TRPV4'.split()) %}
{% set gs3 = '\n'.join('OPRD1 TRPV1 TRPA1 SCN9A OPRM1 TRPM8 TACR1 OPRK1 TAC1 SCN3B KCNS1 TRPV3 TRPV4 CACNA1B CACNA2D2 SCN11A NTRK1 PENK SCN1B OPRL1 PDYN TRPV2 HTR3C HTR3A COMT P2RX3 TRPM5 DRD2 NGFR FAAH ASIC3 PNOC HTR3B TRPM4 CACNA2D3 BDKRB1 ASIC4 HTR2A KCNC2 CHRM4 TRPM3 HTR3E CACNG2 CHRNA7 SCN10A'.split()) %}
{% set gs = TabField(
    name='gs_type',
    label='Gene Sets',
    choices={
        'Upload': [
            MultiFileField(
                name='gs',
                label='Gene Set Files',
                default=['example.gmt'],
                examples={
                    'example.gmt': url_for('static', filename = 'Geneshot_PainGenes.gmt'),
                    'example-1.txt': url_for('static', filename = 'A_Geneshot_PainGenes_GeneRIF_AssociatedGenes.tsv'),
                    'example-2.txt': url_for('static', filename = 'B_Geneshot_PainGenes_AutoRIF_AssociatedGenes.tsv'),
                    'example-3.txt': url_for('static', filename = 'C_Geneshot_PainGenes_GeneRIF_PredictedGenes_AutoRIF-CoOccurrence.tsv'),
                },
            )
        ],
        'Paste': [
            VariableField(
                name='gs_text',
                label = 'Gene Sets',
                default=[gs1, gs2, gs3],
                min=2,
                max=6,
                field=TextListField(
                    name = 'gs_textlist',
                    label = 'Gene Set',
                    examples={
                        'example 1': gs1,
                        'example 2': gs2,
                        'example 3': gs3,
                    },
                ),
            )
        ],
    },
    section = 'section1',
) %}

gs = {{ gs.value[0] }}

venndiagram = {{ BoolField(name = 'venndiagram', label = 'Venn Diagram?', default = 'true', description = 'Select \'Yes\' if you would like to generate a Venn diagram. Otherwise, select \'No\'', section = 'section2') }}

scheme = "{{ ChoiceField(name = 'scheme', label = 'Color Scheme', choices = ['viridis', 'cool', 'plasma', 'inferno', 'magma'], default = 'viridis', description = 'Choose a color scheme for your Venn diagram', section = 'section2') }}"

venn_file_format = {{ MultiCheckboxField(name = 'venn_file_format', label = 'File Format', choices = ['png', 'jpg', 'svg'], default = ['png'], description = 'Select the format(s) to save your Venn diagram', section = 'section2') }}

venn_file_name = {{ StringField(name = 'venn_file_name', label = 'File Name', default = 'venn', description = 'Enter a name/description to save your Venn diagram', section = 'section2') }}

svenn = {{ BoolField(name = 'svenn', label = 'SuperVenn?', default = 'true', description = 'Select \'Yes\' if you woul dlike to generate a SuperVenn diagram. Otherwise, select \'No\'', section = 'section3') }}

annotations = {{ IntField(name = 'annotations', label = 'Minimum Intersection Size to be Displayed', default = 1, min = 1, description = 'If you are comparing many sets, displaying all the intersection sizes can make the figure cluttered. Any intersection size below this value will not be displayed.', section = 'section3') }}

svenn_file_format = {{ MultiCheckboxField(name = 'svenn_file_format', label = 'File Format', choices = ['png', 'jpg', 'svg'], default = ['png'], description = 'Select the format(s) to save your Venn diagram', section = 'section3') }}

svenn_file_name = {{ StringField(name = 'svenn_file_name', label = 'File Name', default = 'supervenn', description = 'Enter a name/description to save your Venn diagram', section = 'section3') }}


upset = {{ BoolField(name = 'upset', label = 'UpSet Plot?', default = 'true', description = 'Select \'Yes\' if you would like to generate an UpSet plot. Otherwise, select \'No\'', section = 'section4') }}

orient = "{{ ChoiceField(name = 'orient', label = 'Orientation', choices = ['Horizontal', 'Vertical'], default = 'Horizontal', description = 'Choose whether your UpSet plot will be displayed horizontally or vertically', section = 'section4') }}"

color = "{{ ChoiceField(name = 'color', label = 'Color', choices = ['Black', 'Blue', 'Red', 'Green', 'Grey', 'Orange', 'Purple', 'Yellow', 'Pink'], default = 'Black', section = 'section4') }}"

counts = {{ BoolField(name = 'counts', label = 'Show Counts?', default = 'true', description = 'This labels the intersection size bars with the cardinality of the intersection.', section = 'section4') }}

percent = {{ BoolField(name = 'percent', label = 'Show Percentages?', default = 'false', description = 'This labels the intersection size bars with the percentage of the intersection relative to the total dataset.', section = 'section4') }}

figure_file_format = {{ MultiCheckboxField(name = 'figure_file_format', label = 'File Format', choices = ['png', 'jpg', 'svg'], default = ['png'], description = 'Select the format to save your figure', section = 'section4') }}

output_file_name = {{ StringField(name = 'output_file_name', label = 'File Name', default = 'UpSet_plot', description = 'Enter a name/description to save your UpSet Plot', section = 'section4') }}

background = {{ IntField(name = 'background', label = 'Background', default = 20000, description = 'Human genes typically have a background of 20,000', section = 'section5') }}

significance = {{ ChoiceField(name = 'significance', label = 'Significance Level', choices = {'0.01': '0.01', '0.05': '0.05', '0.10': '0.10'}, default = '0.05', description = 'Choose a significance level', section = 'section5')}}

final_venn_file_names = [str(venn_file_name + '.' + file_type) for file_type in venn_file_format]

final_svenn_file_names = [str(svenn_file_name + '.' + file_type) for file_type in svenn_file_format]

final_output_file_names = [str(output_file_name + '.' + file_type) for file_type in figure_file_format]

In [ ]:
#Color for UpSet plot
color_conversion = {
    'Black': 'black',
    'Blue': 'lightskyblue',
    'Red': 'tomato',
    'Green': 'mediumspringgreen',
    'Grey': 'lightgrey',
    'Orange': 'orange',
    'Purple': 'plum',
    'Yellow': 'yellow',
    'Pink': 'lightpink'
}

color = color_conversion[color]

In [ ]:
# Displaying Figures
def figure_title(label, title):
    display(HTML(f"<div style='font-size:2rem; padding;1rem 0;'><b>{label}</b>: {title}</div>"))

def figure_legend(label, title, content=""):
    display(HTML(f"<div><b>{label}</b>: <i>{title}</i>. {content} </div>"))

# Saving Figures
def save_figure(plot_name, **kwargs):
    import io
    mem = io.BytesIO()
    pyplot.savefig(mem, bbox_inches='tight')
    with open(plot_name, 'wb') as fw:
        fw.write(mem.getbuffer())

# Loading helper
def load_sets(*files):
    ''' Load a set of files into pairs of labeled sets
    '''
    sets = {}
    for file in map(pathlib.Path, files):
        if file.suffix == '.gmt':
            for line in map(str.strip, file.open('r')):
                line_split = line.split('\t')
                if len(line_split) < 3: continue
                term, description, *geneset = line_split
                term_description = ' '.join(filter(None, map(str.strip, [
                    file.stem if len(files) > 1 else '',
                    term,
                    description,
                ])))
                sets[term_description] = set(filter(None, map(str.strip, geneset)))
        else:
            # assumed file is newline separated genes if not a gmt
            sets[file.stem] = set(map(str.strip, file.open('r')))
    return sets

In [ ]:
%%appyter code_exec
# Add the appropriate gene lists to the dictionary

{% if gs.raw_value == 'Upload' %}
gsdict = load_sets(*gs)
{% elif gs.raw_value == 'Paste' %}
gsdict = { str(i): set(s) for i, s in enumerate(gs) }
{% endif %}


## Venn Diagram

In [ ]:
# Generate the venn diagram

if venndiagram:
    venn(gsdict, cmap = scheme)

    for plot_name in final_venn_file_names:
        save_figure(plot_name)
    
    figure_title("Figure 1", "Venn diagram")
    pyplot.show()
    figure_legend("Figure 1", "Venn diagram", "This Venn diagram compares the inputted gene sets and displays the intersections between them.")
    

    if len(gsdict) == 6:
        pseudovenn(gsdict)

In [ ]:
# Download Venn Diagrams
for i, file in enumerate(final_venn_file_names):
    display(FileLink(file, result_html_prefix=str('Download ' + venn_file_format[i] + ': ')))

## SuperVenn Diagram

In [ ]:
# SuperVenn
if svenn:
    figure_title("Figure 2", "SuperVenn")
    supervenn(list(gsdict.values()), list(gsdict.keys()), sets_ordering= 'minimize gaps', widths_minmax_ratio=0.1, min_width_for_annotation=annotations)
    for plot_name in final_svenn_file_names:
        save_figure(plot_name)
    figure_legend("Figure 2", "SuperVenn", "The numbers on the right represent the set sizes and the numbers on the top show how many sets the intersection is part of. The overlapping portions of the colored bars correspond to set intersections.")

In [ ]:
# Download Venn Diagrams
for i, file in enumerate(final_svenn_file_names):
    display(FileLink(file, result_html_prefix=str('Download ' + svenn_file_format[i] + ': ')))

## UpSet Plot

In [ ]:
# UpSet Plots
if upset:
    df = from_contents(gsdict)
    plot(df, orientation = orient.lower(), facecolor = color, show_counts = counts, show_percentages = percent)
    for plot_name in final_output_file_names:
        save_figure(plot_name)
    
    figure_title("Figure 3", "UpSet Plot")
    pyplot.show()
    figure_legend("Figure 3", "UpSet Plot", "This UpSet plot displays the set intersections as a matrix with the cardinalities shown as bars.")
    

In [ ]:
# Download UpSet Plots
for i, file in enumerate(final_output_file_names):
    display(FileLink(file, result_html_prefix = str('Download ' + figure_file_format[i] + ': ')))

## List of Set Intersections

In [ ]:
#Linking to Enrichr
def enrichr_link(gene_list):
    ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/addList' 
    genes_str = '\n'.join(gene_list)
    description = 'Example Gene List'
    payload = {
        'list': (None, genes_str),
        'description': (None, description)
    }

    response = requests.post(ENRICHR_URL, files=payload)
    if not response.ok:
        raise Exception('Error analyzing gene list')

    time.sleep(0.5)

    data = json.loads(response.text)

    short_id = data['shortId']
    return [str(short_id)]


In [ ]:
def get_venn_sections(sets):
    num_combinations = 2 ** len(sets)
    bit_flags = [2 ** n for n in range(len(sets))]
    flags_zip_sets = [z for z in zip(bit_flags, sets)]

    combo_sets = []
    for bits in range(num_combinations - 1, 0, -1):
        include_sets = [s for flag, s in flags_zip_sets if bits & flag]
        exclude_sets = [s for flag, s in flags_zip_sets if not bits * flag]
        combo = set.intersection(*include_sets)
        combo = set.difference(combo, *exclude_sets)
        tag = ''.join([str(int((bits & flag) > 0)) for flag in bit_flags])
        combo_sets.append((tag, combo))
    return combo_sets

In [ ]:
sets = list(gsdict.values())

In [ ]:
def generate_name(combos):
    tag_list = []
    for pair in combos:
        bits = pair[0]
        inter = '('
        diff = '('
        for i in range(len(bits)):
            j = i+1
            set_name = 'Set ' +  str(j)
            if bits[i] == '1':
                inter += set_name
                inter += ' & '
            else:
                diff += set_name
                diff += ' U '

        final_inter = inter[:-3]
        final_inter += ')'
        final_diff = diff[:-3]
        final_diff += ')'

        if final_diff != ')':
            final_name = final_inter + ' - ' + final_diff
        else:
            final_name = final_inter[1:-1]
        tag_list.append(final_name)
    return tag_list

In [ ]:
# Generates visibility booleans for dropdown menu

def generate_visibility(options):
    bools = []
    temp = []
    for x in range (len(options)):
        temp.append(False)
    for x in range(len(options)):
        visible = temp.copy()
        visible[x] = True
        bools.append(visible)
    
    return bools

In [ ]:
# Creates the options for the dropdown menu
def make_options(tuples, names):
    bools = generate_visibility(tuples)
    dropdown = []
    for x in range (len(tuples)):
        option = dict(
            args = [{'visible': bools[x]}],
            label = names[x],
            method = "update"
        )
        dropdown.append(option)
    return dropdown

In [ ]:
def create_enrichr_link(l1):
    results = enrichr_link(l1)
    final_str = str('https://maayanlab.cloud/Enrichr/enrich?dataset='+ results[0])
    return final_str

In [ ]:
# Add Enrichr Links
def add_links():
    l1 = []
    for pair in get_venn_sections(sets):
        if len(pair[1]) >= 5:
            temp = pair
            new_tuple = temp + tuple(create_enrichr_link(pair[1]).split(' '))
            l1.append(new_tuple)
    return l1

new_venn_sections = add_links()

In [ ]:
def set_to_list(l1):
    l2 = []
    l2.append('Size: ' + str(len(list(l1[1]))))
    l2.append('Access your complete Enrichment results here: <a href="' + str(l1[2]) + '">' + str(l1[2]) + '</a>')
    for elem in l1[1]:
        l2.append(elem)
    return l2

In [ ]:
# Create Figure for Set Intersection Item Dropdown
fig = go.Figure()

for pair in new_venn_sections:
    fig.add_trace(
        go.Table(
            header = dict(
                values = ['Intersection Listing'],
                line_color = '#001C55',
                fill_color = '#001C55',
                align = ['left', 'center'],
                font=dict(color='white', size=16)
            ),
            cells = dict(
                values = [set_to_list(pair)],
                line_color = 'white',
                fill_color = '#f5f5f5',
                align = ['left', 'center'],
                font = dict(color = 'darkslategray', size = 14)
            )
        )
    )
## Make Dropdown

fig.update_layout(
    updatemenus = [
        dict(
        buttons=list(make_options(get_venn_sections(sets), generate_name(get_venn_sections(sets)))),
        direction = "down",
        pad = {"r": 10, "t": 10},
        showactive = True,
        x = 0,
        xanchor = "left",
        y = 1.2,
        yanchor = "top"
    ),
    ]
)

figure_title("Table 1", "List of Set Intersections")
fig.show()
display(HTML(f"<div><b>Explanation of Symbols</b>: <br><i>A - B </i> - subtraction of set B from set A</br> <br><i>A & B </i> - intersection of sets A and B</br> <br><i>A U B </i> - union of sets A and B</br> </div>"))
figure_legend("Table 1", "List of Set Intersections", "This table shows the elements contained in each set intersection. A link to Enrichr for further enrichment analysis is provided. Various intersections can be found using the dropdown menu.")

## Fisher's Exact Test Calculations

In [ ]:
# Pair the Gene Sets
matching = []

gene_sets = list(gsdict.keys())
for i in range (len(gene_sets)-1):
    for j in range (i+1, len(gene_sets)):
        matching.append((gene_sets[i], gene_sets[j]))

In [ ]:
# Generates values for Fisher's Exact Test

def generate_values(s1, s2):
    inter = len(s1 & s2)
    not_A_not_B = background - (len(s1) + len(s2) - inter)
    in_A_not_B = len(s1) - inter
    not_A_in_B = len(s2) - inter
    total = not_A_not_B + not_A_in_B + in_A_not_B + inter

    oddsratio, pvalue = stats.fisher_exact([[not_A_not_B, in_A_not_B], [not_A_in_B, inter]])

    if pvalue < significance:
        sig_result = 'This result is <b>significant</b> at p < ' + str(significance)
    else:
        sig_result = 'This result is <b>not significant</b> at p < ' + str(significance)

    values1 = [['<b>Not in B</b>', '<b>In B</b>', '<b>Marginal Column Totals</b>', '<b>p-value</b>: ' + "{:.3e}".format(pvalue), '<b>Odds Ratio</b>: ' + str(oddsratio), '<b>Result</b>: ' + sig_result], [not_A_not_B, not_A_in_B, not_A_not_B+not_A_in_B], [in_A_not_B, inter, in_A_not_B+inter], [not_A_not_B+in_A_not_B, not_A_in_B+inter, str(total) + ' (Grand Total)']]

    return values1

In [ ]:
# Create figure and adds all tables
fig2 = go.Figure()

for pair in matching:
    fig2.add_trace(
        go.Table(
            header = dict(
                values = ['', '<b>Not in A</b>', '<b>In A</b>', '<b>Marginal Row Totals</b>'],
                line_color = '#001C55',
                fill_color = '#001C55',
                align = ['left', 'center'],
                font=dict(color='white', size=12)
            ),
            cells = dict(
                values = generate_values(gsdict[pair[0]], gsdict[pair[1]]),
                line_color = 'white',
                fill_color = [['#f5f5f5', '#f5f5f5', '#f5f5f5', 'white', 'white', 'white']*4],
                align = ['left', 'center'],
                font = dict(color = 'darkslategray', size = 11)
            )
        )
    )

In [ ]:
# Generates names for dropdown menu
def generate_names():
    names = []
    for pair in matching:
        s = pair[0] + ' & ' + pair[1]
        names.append(s)
    return names

In [ ]:
# Generates figure with dropdown menu
names = generate_names()

fig2.update_layout(
    updatemenus = [
        dict(
            buttons=list(make_options(matching, names)),
            direction = "down",
            pad = {"r": 10, "t": 10},
            showactive = True,
            x = 0,
            xanchor = "left",
            y = 1.2,
            yanchor = "top"
        ),
    ]
)

figure_title("Table 2", "Fisher's Exact Test")

fig2.show()

figure_legend("Table 2", "Fisher's Exact Test", "This table shows the results of Fisher's Exact Test. Using the items in the contigency table, the p-value and odds ratio is calculated. The p-value is then compared against the desired significance level. The overlap between various sets can be seen using the dropdown menu.")

## Heatmap of Fisher's Exact Test Results

In [ ]:
def reverse(tuples):
    new_tup = ()
    for k in reversed(tuples):
        new_tup += (k, )
    return new_tup

In [ ]:
def check_sig(s1, s2):
    inter = len(s1 & s2)
    not_A_not_B = background - (len(s1) + len(s2) - inter)
    in_A_not_B = len(s1) - inter
    not_A_in_B = len(s2) - inter
    total = not_A_not_B + not_A_in_B + in_A_not_B + inter

    oddsratio, pvalue = stats.fisher_exact([[not_A_not_B, in_A_not_B], [not_A_in_B, inter]])

    if pvalue == 0:
        return pvalue
    else:
        num = -math.log(pvalue, 10)
        return num

In [ ]:
def heatmap_values(gene_sets):
    values = []
    x_axis = gene_sets
    y_axis = gene_sets
    for i in range (len(x_axis)):
        row = []
        for j in range (len(y_axis)):
            t = (x_axis[i], y_axis[j])
            if t in matching:
                row.append(check_sig(gsdict[x_axis[i]], gsdict[y_axis[j]]))
            elif reverse(t) in matching:
                row.append(check_sig(gsdict[y_axis[j]], gsdict[x_axis[i]]))
            else:
                row.append(None)
        values.append(row)
    return values

In [ ]:
fig3 = go.Figure(data = go.Heatmap(
                    z = heatmap_values(gene_sets),
                    x = gene_sets,
                    y = gene_sets,
                    hoverongaps = False))
figure_title("Figure 4", "Heatmap of Fisher's Exact Test Results")
fig3.show()
figure_legend("Figure 4", "Heatmap of Fisher's Exact Test Results", "This figure displays the results of all Fisher's Exact Tests calculated. The -log(p-values) is shown in the heatmap. Each axis displays which sets are being compared and sets that cannot be compared are given a value of None.")